In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, matthews_corrcoef, roc_auc_score, roc_curve
from sklearn.metrics import accuracy_score
#from sklearn.datasets import load_iris
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.initializers import he_uniform
from keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [ ]:
# Load data from CSV file or other source
file_path = r"F:\other student data\BKUC MS\BKUC Bilal\pca80.csv"
data = pd.read_csv(file_path)

# Assuming the last column is the target variable and all other columns are features
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Define a traditional machine learning model (Random Forest as an example)
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Evaluate the traditional model
y_pred_rf = rf_model.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Random Forest Accuracy: {accuracy_rf:.2f}")


In [ ]:
# Generate predictions or use features from the Random Forest model
X_train_rf = rf_model.predict_proba(X_train)
X_test_rf = rf_model.predict_proba(X_test)


In [ ]:

# Define a simple Neural Network using Keras Sequential API
deep_model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_rf.shape[1],), kernel_initializer=he_uniform()),
    Dropout(0.2),  # Adding dropout with rate 0.2
    Dense(64, activation='relu', kernel_initializer=he_uniform()),
   # Dropout(0.4),  # Adding dropout with rate 0.2
    Dense(32, activation='relu', kernel_initializer=he_uniform()),
   # Dropout(0.3), 
    Dense(16, activation='relu', kernel_initializer=he_uniform()),
   # Dropout(0.2), 
    Dense(1, activation='sigmoid')
])

# Compile the model with Adam optimizer and custom learning rate
adam_optimizer = Adam(learning_rate=0.001)  # Specify learning rate
deep_model.compile(optimizer=adam_optimizer,
                   loss='binary_crossentropy',
                   metrics=['accuracy'])

# Train the model using features from Step 1 (Random Forest predictions)
history = deep_model.fit(X_train_rf, y_train,
                         epochs=100,
                         batch_size=32,
                         validation_data=(X_test_rf, y_test),
                         verbose=1)


In [ ]:
# Evaluate the Sequential Hybrid Model on test data
y_pred_deep = deep_model.predict(X_test_rf)
y_pred_deep_classes = np.argmax(y_pred_deep, axis=1)  # Convert probabilities to class labels
accuracy_deep = accuracy_score(y_test, y_pred_deep_classes)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, mean_squared_error, mean_absolute_error
from sklearn import metrics

# Evaluate the Sequential Hybrid Model on test data
y_pred_deep = deep_model.predict(X_test_rf)
y_pred_deep_classes = (y_pred_deep > 0.5).astype(int)  # Convert probabilities to binary class labels

# Accuracy
accuracy_deep = accuracy_score(y_test, y_pred_deep_classes)
print(f"Sequential Hybrid Model Accuracy: {accuracy_deep:.2f}")

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_deep_classes)
print("Confusion Matrix:")
print(cm)

# Sensitivity (Recall)
sensitivity = recall_score(y_test, y_pred_deep_classes)
print(f"Sensitivity: {sensitivity:.2f}")

# Specificity
specificity = metrics.recall_score(y_test, y_pred_deep_classes, pos_label=0)
print(f"Specificity: {specificity:.2f}")

# F1 Score
f1 = f1_score(y_test, y_pred_deep_classes)
print(f"F1 Score: {f1:.2f}")

# Matthews Correlation Coefficient (MCC)
mcc = metrics.matthews_corrcoef(y_test, y_pred_deep_classes)
print(f"MCC: {mcc:.2f}")

# Precision
precision = precision_score(y_test, y_pred_deep_classes)
print(f"Precision: {precision:.2f}")

# Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred_deep_classes)
print(f"Mean Squared Error (MSE): {mse:.2f}")

# Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred_deep_classes)
print(f"Mean Absolute Error (MAE): {mae:.2f}")

# Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")

# P-test and T-test (you need to define how you want to perform these tests based on your hypothesis)
# For example:
# p_value = ...
# t_statistic = ...

# Print or use p_value and t_statistic as needed
# print(f"P-Test: {p_value:.2f}")
# print(f"T-Test: {t_statistic:.2f}")


In [ ]:
# Plotting training & validation accuracy
plt.figure(figsize=(10, 6))
plt.plot(history.history['accuracy'], label='Train Accuracy', linewidth=3)
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=3)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Model Accuracy')
plt.grid(True, linestyle='--', alpha=0.7)
plt.savefig("Model Accuracy.png", bbox_inches='tight', transparent=False)
# Show plot
plt.show()

In [ ]:
import os
import matplotlib.pyplot as plt

# Get the user's home directory
home_dir = os.path.expanduser('~')

# Specify the save path
save_path = os.path.join(home_dir, 'model_loss_plot.png') #, transparent=False

plt.figure(figsize=(8, 4))
plt.plot(history.history['loss'], label='Train Loss', linewidth=3)
plt.plot(history.history['val_loss'], label='Test Loss', linewidth=3)
plt.xlabel('Epochs', fontsize=14)
plt.ylabel('Loss (%)', fontsize=14)
plt.legend(fontsize=12)
#plt.title('Model Loss', fontsize=16)
plt.grid(True, linestyle='--', linewidth=0.5)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.grid(True, linestyle='--', alpha=0.7)

plt.show()


In [ ]:
from sklearn.decomposition import PCA

# Assuming you have PCA applied after preprocessing steps
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Plot PCA visualization
plt.figure(figsize=(8, 6))
plt.scatter(X_train_pca[:, 0], X_train_pca[:, 1], c=y_train, cmap='viridis', edgecolor='k', s=50, alpha=0.7)
plt.title('PCA of Training Data')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.colorbar()
plt.grid(True, linestyle='--', alpha=0.7)
plt.savefig("PCA of Training Data.png", bbox_inches='tight', transparent=False)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve

# Predict probabilities for positive class
y_pred_deep_proba = deep_model.predict(X_test_rf)

precision, recall, thresholds = precision_recall_curve(y_test, y_pred_deep_proba)

plt.figure(figsize=(8, 6))
plt.plot(recall, precision, color='b', marker='o', markersize=5, label='Precision-Recall Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve - Sequential Hybrid Model')
plt.legend(loc='best')
plt.grid(True, linestyle='--', alpha=0.7)
plt.savefig("Precision-Recall Curve.png", bbox_inches='tight', transparent=False)
plt.show()


In [ ]:
def plot_learning_curve(history):
    plt.figure(figsize=(10, 6))
    
    # Plotting with different line styles, markers, and widths
    plt.plot(history.history['accuracy'], label='Train Accuracy', linestyle='-.', linewidth=2, marker='o')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy', linestyle='--', linewidth=2, marker='^')
    plt.plot(history.history['loss'], label='Train Loss', linestyle='-.', linewidth=2, marker='s')
    plt.plot(history.history['val_loss'], label='Validation Loss', linestyle=':', linewidth=2, marker='x')
    
    plt.xlabel('Epochs')
    plt.ylabel('Metrics')
    plt.legend()
    plt.title('Learning Curve')
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.savefig("Learning Curve.png", bbox_inches='tight', transparent=False)
    
    plt.show()

# Assuming `history` is your Keras history object
plot_learning_curve(history)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming rf_model, data, and indices are defined as in your context

# Extract feature importances from Random Forest model
feature_importances = rf_model.feature_importances_
feature_names = data.columns[:-1]  # Assuming the last column is the target variable

# Sort feature importances in descending order
indices = np.argsort(feature_importances)[::-1]

plt.figure(figsize=(12, 8))

# Define a colormap
colors = plt.cm.viridis(feature_importances / max(feature_importances))

# Plot bars
bars = plt.bar(range(len(indices)), feature_importances[indices], align='center', color=colors)

# Customize ticks and labels
plt.xticks(range(len(indices)), feature_names[indices], rotation=90)
plt.gca().set_xticks(range(min(len(indices), 100)))  # Show up to top 67 features
plt.gca().set_xticklabels(feature_names[indices][:0], rotation=90)

# Add labels and title
plt.xlabel('Feature')
plt.ylabel('Importance')
#plt.title('Feature Importance - Random Forest')

# Create a ScalarMappable object
mappable = plt.cm.ScalarMappable(cmap='viridis')
mappable.set_array(feature_importances)

# Add color bar with specified Axes
cbar = plt.colorbar(mappable, ax=plt.gca())
cbar.set_label('Importance')

plt.tight_layout()
plt.grid(True, linestyle='--', alpha=0.7)
plt.savefig("Feature Importance.png", bbox_inches='tight', transparent=False)
plt.show()


In [ ]:
plt.figure(figsize=(8, 6))
plt.hist(y, bins=len(np.unique(y)), edgecolor='k')
plt.xlabel('Class')
plt.ylabel('Count')
plt.title('Class Distribution')
plt.grid(True, linestyle='--', alpha=0.7)
plt.savefig("Class Distribution.png", bbox_inches='tight', transparent=False)
plt.show()


In [ ]:
# Assuming only two features after PCA
if X_train_pca.shape[1] == 2:
    plt.figure(figsize=(8, 6))
    x_min, x_max = X_train_pca[:, 0].min() - 1, X_train_pca[:, 0].max() + 1
    y_min, y_max = X_train_pca[:, 1].min() - 1, X_train_pca[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1),
                         np.arange(y_min, y_max, 0.1))
    Z = deep_model.predict(np.c_[xx.ravel(), yy.ravel()])[:, 0]
    Z = Z.reshape(xx.shape)
    plt.contourf(xx, yy, Z, alpha=0.8)
    plt.scatter(X_train_pca[:, 0], X_train_pca[:, 1], c=y_train, cmap='viridis', edgecolor='k', s=50, alpha=0.7)
    plt.title('Decision Boundary - Sequential Hybrid Model')
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.colorbar()
  
    plt.savefig("Decision Boundary.png", bbox_inches='tight', transparent=False)
    plt.show()
else:
    print("Cannot plot decision boundary. Ensure PCA reduces data to 2 dimensions.")


In [ ]:
from sklearn.calibration import calibration_curve

# Compute calibration curve
prob_true, prob_pred = calibration_curve(y_test, deep_model.predict(X_test_rf)[:, 0], n_bins=10)

plt.figure(figsize=(8, 6))
plt.plot(prob_pred, prob_true, marker='o', linewidth=3, label='Calibration Curve')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Perfectly Calibrated')
plt.xlabel('Predicted Probability')
plt.ylabel('True Probability')
plt.title('Calibration Curve')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)
plt.savefig("Calibration Curve.png", bbox_inches='tight', transparent=False)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming regression task and `y_test` and `y_pred_deep` are continuous values
if np.issubdtype(y_test.dtype, np.number) and np.issubdtype(y_pred_deep.dtype, np.number):
    residuals = y_test - y_pred_deep.flatten()

    plt.figure(figsize=(8, 4))
    
    # Customizing scatter plot with different colors and markers
    plt.scatter(y_pred_deep, residuals, c=residuals, cmap='coolwarm', edgecolor='k', s=500, alpha=0.7)
    
    # Adding color bar to indicate residual values
    plt.colorbar(label='Residual')
    
    plt.xlabel('Predicted Values')
    plt.ylabel('Residuals')
    plt.title('Residual Plot')
    plt.axhline(y=0, color='r', linestyle='--')
    plt.show()
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.savefig("Residual Plot.png", bbox_inches='tight', transparent=False)
else:
    print("Cannot plot residuals. Ensure `y_test` and `y_pred_deep` are continuous values.")


In [ ]:
# from scikitplot.metrics import plot_cumulative_gain

# plt.figure(figsize=(8, 6))
# plot_cumulative_gain(y_test, deep_model.predict_proba(X_test_rf))
# plt.title('Cumulative Gain Curve')
# plt.show()


In [ ]:
# from scikitplot.metrics import plot_lift_curve

# plt.figure(figsize=(8, 6))
# plot_lift_curve(y_test, deep_model.predict_proba(X_test_rf))
# plt.title('Lift Curve')
# plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import validation_curve
from sklearn.ensemble import RandomForestClassifier

# Example of validation curve for max_depth in a decision tree
param_range = np.arange(1, 20, 2)
train_scores, test_scores = validation_curve(
    RandomForestClassifier(), X_train, y_train, param_name="max_depth", param_range=param_range,
    scoring="accuracy", n_jobs=-1, verbose=1)

train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

plt.figure(figsize=(10, 6))
plt.title("Validation Curve with Random Forest")
plt.xlabel("Max Depth")
plt.ylabel("Score")
plt.ylim(0.0, 1.1)

# Define colors and line widths
plt.semilogx(param_range, train_scores_mean, label="Training score", color="orange", marker='o', markersize=8)
plt.fill_between(param_range, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.2,
                 color="orange", lw=2)

plt.semilogx(param_range, test_scores_mean, label="Cross-validation score", color="blue", marker='s', markersize=8)
plt.fill_between(param_range, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.2,
                 color="blue", lw=2)

plt.legend(loc="best")
plt.grid(True, linestyle='--', alpha=0.7)
plt.savefig("Cross-validation score.png", bbox_inches='tight', transparent=False)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import validation_curve
from sklearn.ensemble import RandomForestClassifier

# Example of validation curve for max_depth in a decision tree
param_range = np.arange(1, 20, 2)
train_scores, test_scores = validation_curve(
    RandomForestClassifier(), X_train, y_train, param_name="max_depth", param_range=param_range,
    scoring="accuracy", n_jobs=-1, verbose=1)

train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

plt.figure(figsize=(10, 6))
plt.title("Validation Curve with Random Forest")
plt.xlabel("Max Depth")
plt.ylabel("Score")
plt.ylim(0.0, 1.1)

# Plot training scores
plt.semilogx(param_range, train_scores_mean, label="Training score", color="darkorange", lw=2)
plt.fill_between(param_range, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.2, color="darkorange", lw=2)

# Plot cross-validation scores
plt.semilogx(param_range, test_scores_mean, label="Cross-validation score", color="navy", lw=2)
plt.fill_between(param_range, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.2, color="navy", lw=2)

plt.legend(loc="best")
plt.grid(True, linestyle='--', alpha=0.7)
plt.savefig("Cross-validation score.png", bbox_inches='tight', transparent=False)
plt.show()


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
import matplotlib.pyplot as plt
import numpy as np

# Generate a sample dataset (replace with your actual data loading)
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=1000, n_features=20, random_state=42)

# Example of comparing multiple classifiers using cross-validation scores
models = [
    ('Random Forest', RandomForestClassifier()),
    ('Gradient Boosting', GradientBoostingClassifier()),
    ('SVM', SVC(probability=True)),
    ('AdaBoost', AdaBoostClassifier())
]

results = []
names = []
for name, model in models:
    cv_results = cross_val_score(model, X, y, scoring='accuracy')
    results.append(cv_results)
    names.append(name)
    print(f"{name}: Mean Accuracy = {np.mean(cv_results):.2f}, Std = {np.std(cv_results):.2f}")

plt.figure(figsize=(10, 6))
plt.boxplot(results, labels=names)
plt.title('Model Comparison')
plt.ylabel('Accuracy')
plt.grid(True, linestyle='--', alpha=0.7)
plt.savefig("Model Comparison.png", bbox_inches='tight', transparent=False)
plt.show()


In [ ]:
# Example of visualizing word embeddings in a deep learning model
from tensorflow.keras.layers import Embedding

# Assuming `embedding_layer` is an Embedding layer in `deep_model`
if 'deep_model' in locals():
    for layer in deep_model.layers:
        if isinstance(layer, Embedding):
            embeddings = layer.get_weights()[0]
            plt.figure(figsize=(10, 8))
            for i in range(embeddings.shape[0]):
                plt.text(embeddings[i, 0], embeddings[i, 1], str(i))
            plt.title('Embedding Visualization')
            plt.xlabel('Dimension 1')
            plt.ylabel('Dimension 2')
            plt.show()
            break
else:
    print("Deep learning model (`deep_model`) not found. Please ensure it is defined and trained.")
